# Test metrics on Toy Example

In [29]:
# FUNCTIONS / CLASSES :
import sys
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
sys.path.append("./code")
from metrics import METEOR, BLEU, NIST, WACC, compute_metrics
from data_processing import WMT22, dataset

metrics = [METEOR, BLEU, NIST, WACC]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
reference = "Je me présente je m'appelle Henri"
good_candidate = "Je me présente je me prénomme Henri"
avg_candidate = "Bonjour mon nom est Henri"
bad_candidate = "J'aime les pizzas"

## 1 - METEOR

In [47]:
print(METEOR(good_candidate, reference))
print(METEOR(avg_candidate, reference))
print(METEOR(bad_candidate, reference))

0.5391304347826087
0.09803921568627452
0.0


## 2 - BLEU

In [48]:
print(BLEU(good_candidate, reference))
print(BLEU(avg_candidate, reference))
print(BLEU(bad_candidate, reference))

0.7054014205932617
0.1666666567325592
0.0


## 3 - NIST

In [49]:
print(NIST(good_candidate, reference))
print(NIST(avg_candidate, reference))
print(NIST(bad_candidate, reference))

1.9656602059902528
0.3869880158145604
0.0


## 4 - WAcc

In [50]:
print(WACC(good_candidate, reference))
print(WACC(avg_candidate, reference))
print(WACC(bad_candidate, reference))

0.7142857015132904
0.0
-1.0


## Test on WMT Data

In [69]:
df = WMT22.read_data()

In [70]:
sample = df.sample(1000)
sample.sample(2)

,seg_id,sys,hyp,domain,doc,source,ref,score,sltl
29200,911,QUARTZ_TuneReranking,Возврат товара должен быть произведен в течени...,ecommerce,ecommerce_en_24,Returns must be within 30 days of delivery in ...,Возвраты осуществляются в течение 30 дней с мо...,0.0,enru
1085,1102,bleu_bestmbr,"Через несколько недель после того, как я верну...",social,t1_hoh4p6r,"A few weeks after I returned home, I received ...","Спустя несколько недель после того, как я верн...",-3.0,enru


In [78]:
for metric in tqdm(metrics):
    
    sample[str(metric.__name__)] = sample.apply(lambda x : metric(
        reference = x.ref, 
        candidate = x.hyp,), axis = 1)

100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.94it/s]


In [79]:
sample[["score"] + [str(metric.__name__) for metric in metrics]].corr(method = "spearman")

,score,METEOR,BLEU,NIST,WACC
score,1.000000,0.129131,0.107183,0.035658,0.102204
METEOR,0.129131,1.000000,0.876642,0.728350,0.762269
BLEU,0.107183,0.876642,1.000000,0.777584,0.889027
NIST,0.035658,0.728350,0.777584,1.000000,0.636838
WACC,0.102204,0.762269,0.889027,0.636838,1.000000


In [81]:
sample[["score"] + [str(metric.__name__) for metric in metrics]].corr(method = "kendall")

,score,METEOR,BLEU,NIST,WACC
score,1.000000,0.100935,0.083766,0.027280,0.079863
METEOR,0.100935,1.000000,0.710016,0.538894,0.590097
BLEU,0.083766,0.710016,1.000000,0.593039,0.732003
NIST,0.027280,0.538894,0.593039,1.000000,0.460533
WACC,0.079863,0.590097,0.732003,0.460533,1.000000


# Test dataset and dataloader

In [27]:
dataset_WMT22 = dataset(set_name = "WMT22")

[nltk_data] Downloading package wordnet to /Users/henriup/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/henriup/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
dataset_WMT22.__getitem__(0)

('Sie können jederzeit zurückkehren, da unser Chat-Service-Fenster rund um die Uhr geöffnet ist',
 'Sie können jederzeit wiederkommen, da unser Chat-Service-Fenster täglich rund um die Uhr geöffnet ist')

In [72]:
metrics_scores = compute_metrics(
    set_name = "WMT22",
    metrics = metrics,
    batch_size = 20,
    path = "./data/metrics_scores.csv"
)

100%|██████████| 5/5 [00:00<00:00, 34.06it/s]


In [73]:
metrics_scores = pd.read_csv(
    "./data/metrics_scores.csv",
    index_col = "gold_score"
)

In [74]:
metrics_scores

,hyp,ref,METEOR,BLEU,NIST,WACC
gold_score,,,,,,
0.0,"Sie können jederzeit zurückkehren, da unser Ch...","Sie können jederzeit wiederkommen, da unser Ch...",0.867122,0.854733,3.555746,0.857143
0.0,"Ich hoffe sehr, dass Sie eine Lösung finden","Ich hoffe wirklich, dass Sie eine Lösung finde...",0.801768,0.772185,2.817808,0.777778
0.0,"Vielen Dank, dass Sie #PRS_ORG# kontaktiert ha...","Vielen Dank, dass Sie #PRS_ORG# kontaktiert ha...",0.689663,0.500000,2.730337,0.466667
0.0,Ich wünsche Ihnen einen schönen Abend.,Ich wünsche Ihnen noch einen schönen Abend.,0.875743,0.846482,2.782738,0.857143
-1.0,Der Iran meldet die niedrigste Anzahl tägliche...,Iran meldet niedrigste Zahl täglicher COVID-19...,0.823621,0.615385,2.569898,0.500000
...,...,...,...,...,...,...
-1.0,"Warten Sie, bis der Bildschirm „Wiederherstell...","Warten Sie, bis der Bildschirm „Wiederherstell...",0.999711,1.000000,3.584963,1.000000
-4.0,Lassen Sie den Netzschalter los.,Lassen Sie die Power-Taste los.,0.625000,0.600000,1.723308,0.600000
-2.0,"Nachdem der E-Reader zurückgesetzt wurde, werd...","Nachdem das Lesegerät zurückgesetzt wurde, wer...",0.881307,0.857143,3.488938,0.857143
